In [2]:
import pandas as pd

In [3]:
# df = pd.read_csv('chaker.csv')

In [4]:
df = pd.read_csv('data.csv')

In [5]:
df.head()

,Concept,paragraph,definitions,similarity,Verbe,noed_connected,key_list
0,plan risk management,plan risk management be the process of define ...,how to conduct risk management activity for a ...,"['risk management plan', 'plan risk response']",be,"['risk management plan', 'project management p...","['risk', 'management', 'plan', 'process', 'sta..."
1,project charter,describe in section 4.1.3.1 . the project char...,high - level project description and boundary,[],document,"['project document', 'project document', 'proj...","['charter', 'section', 'description', 'project..."
2,project management plan,describe in section 4.2.3.1 . in planning proj...,subsidiary management plan,"['project management plan', 'project managemen...",planning,"['risk management plan', 'project management p...","['risk', 'outline', 'planning', 'subsidiary', ..."
3,project document,project document that can be consider as input...,stakeholder,"['project document', 'project document update'...",be,"['project document', 'project document', 'proj...","['stakeholder', 'risk', 'responsibility', 'ove..."
4,enterprise environmental factor,the enterprise environmental factor that can i...,overall risk threshold set by the organization...,"['enterprise environmental factor', 'enterpris...",influence,"['enterprise environmental factor', 'enterpris...","['risk', 'environmental', 'enterprise', 'stake..."


In [6]:
df.fillna(method='bfill', inplace=True)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_1712\3314729575.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)


In [7]:
df.rename(columns={'Concept': 'subject'}, inplace=True)
df.rename(columns={'Verbe': 'predicate'}, inplace=True)
df.rename(columns={'definitions': 'object'}, inplace=True)

In [8]:
df.head()

,subject,paragraph,object,similarity,predicate,noed_connected,key_list
0,plan risk management,plan risk management be the process of define ...,how to conduct risk management activity for a ...,"['risk management plan', 'plan risk response']",be,"['risk management plan', 'project management p...","['risk', 'management', 'plan', 'process', 'sta..."
1,project charter,describe in section 4.1.3.1 . the project char...,high - level project description and boundary,[],document,"['project document', 'project document', 'proj...","['charter', 'section', 'description', 'project..."
2,project management plan,describe in section 4.2.3.1 . in planning proj...,subsidiary management plan,"['project management plan', 'project managemen...",planning,"['risk management plan', 'project management p...","['risk', 'outline', 'planning', 'subsidiary', ..."
3,project document,project document that can be consider as input...,stakeholder,"['project document', 'project document update'...",be,"['project document', 'project document', 'proj...","['stakeholder', 'risk', 'responsibility', 'ove..."
4,enterprise environmental factor,the enterprise environmental factor that can i...,overall risk threshold set by the organization...,"['enterprise environmental factor', 'enterpris...",influence,"['enterprise environmental factor', 'enterpris...","['risk', 'environmental', 'enterprise', 'stake..."


In [9]:
triples = df.apply(lambda row: (row['subject'], row['predicate'], row['object']), axis=1).tolist()


In [10]:
import random
from sklearn.model_selection import train_test_split

# Assuming you have a list of triples (subject, predicate, object)

# Split the triples into subject, predicate, and object lists
subjects, predicates, objects = zip(*triples)

# Split the data into training and testing sets using random split
train_subjects, test_subjects, train_predicates, test_predicates, train_objects, test_objects = train_test_split(subjects, predicates, objects, test_size=0.3, random_state=42)

# Print the training and testing triples
train_triples = list(zip(train_subjects, train_predicates, train_objects))
test_triples = list(zip(test_subjects, test_predicates, test_objects))

print("Training Triples:")
for triple in train_triples:
    print(triple)

print("\nTesting Triples:")
for triple in test_triples:
    print(triple)



Training Triples:
('perform quantitative risk analysis', 'be', 'plan risk response process')
('expert judgment', 'be', 'bias')
('risk categorization', 'be', 'common root cause')
('datum gathering', 'be', 'individual project risk and other source of uncertainty')
('expert judgment', 'be', 'specialized knowledge')
('organizational process asset', 'include', 'project file')
('enterprise environmental factor', 'include', 'industry study of similar project')
('interpersonal and team skill', 'be', 'facilitation')
('data analysis', 'be', 'stakeholder analysis')
('project document update', 'be', 'risk report')
('strategy for overall project risk', 'be', 'avoid')
('datum analysis', 'be', 'root cause analysis')
('interpersonal and team skill', 'be', 'influence')
('project document update', 'be', 'assumption log .')
('representation of uncertainty', 'be', 'probability distribution')
('agreement', 'require', 'milestone date , contract type , acceptance criterion , and award and penalty')
('risk re

# code

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# Define the KnowledgeGraph dataset class
class KnowledgeGraphDataset(Dataset):
    def __init__(self, triples, entity2idx, relation2idx):
        self.triples = triples
        self.entity2idx = entity2idx
        self.relation2idx = relation2idx

    def __len__(self):
        return len(self.triples)

    def __getitem__(self, idx):
        subject, relation, object_ = self.triples[idx]
        return self.entity2idx[subject], self.relation2idx[relation], self.entity2idx[object_]

# Define the TransE model
class TransE(nn.Module):
    def __init__(self, num_entities, num_relations, embedding_dim):
        super(TransE, self).__init__()
        self.num_entities = num_entities
        self.num_relations = num_relations
        self.embedding_dim = embedding_dim

        self.entity_embeddings = nn.Embedding(num_entities, embedding_dim)
        self.relation_embeddings = nn.Embedding(num_relations, embedding_dim)

    def forward(self, sample):
        subject = self.entity_embeddings(sample[:, 0])
        relation = self.relation_embeddings(sample[:, 1])
        object_ = self.entity_embeddings(sample[:, 2])

        return subject, relation, object_

# Create a list of triples (subject, relation, object)

# Map entities and relations to numerical indices
entities = list(set([subject for subject, _, _ in triples] + [object_ for _, _, object_ in triples]))
entity2idx = {entity: idx for idx, entity in enumerate(entities)}
relations = list(set([relation for _, relation, _ in triples]))
relation2idx = {relation: idx for idx, relation in enumerate(relations)}

# Define the KnowledgeGraph dataset
dataset = KnowledgeGraphDataset(triples, entity2idx, relation2idx)



In [12]:
# Define the dataloader
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Set the hyperparameters
num_entities = len(entities)
num_relations = len(relations)
embedding_dim = 3
learning_rate = 0.02
num_epochs = 3

# Define the TransE model
model = TransE(num_entities, num_relations, embedding_dim)

# Define the loss function
criterion = nn.MarginRankingLoss(margin=1.0)

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    for batch in dataloader:
        optimizer.zero_grad()

        # Positive samples
        positive_samples = torch.stack(batch, dim=1).long()

        # Generate negative samples by corrupting the objects in positive samples
        negative_samples = positive_samples.clone()


        negative_samples[:, 2] = torch.randint(num_entities, size=(negative_samples.size(0),))

        # Forward pass
        pos_subject, pos_relation, pos_object = model(positive_samples)
        neg_subject, neg_relation, neg_object = model(negative_samples)

        # Compute the loss
        # Compute the loss
        target = torch.ones(pos_subject.size(0), 1)
        loss = criterion(pos_subject + pos_relation - pos_object, neg_subject + neg_relation - neg_object, target)


        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        print("Epoch: {}, Loss: {:.4f}".format(epoch, loss.item()))




c:\Users\ASUS\anaconda3\envs\PI\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch: 0, Loss: 1.6761
Epoch: 0, Loss: 1.4875
Epoch: 0, Loss: 1.8762
Epoch: 0, Loss: 1.9638
Epoch: 0, Loss: 1.3489
Epoch: 0, Loss: 1.4668
Epoch: 0, Loss: 0.9840
Epoch: 0, Loss: 1.0341
Epoch: 0, Loss: 1.6594
Epoch: 0, Loss: 1.6602
Epoch: 0, Loss: 1.2669
Epoch: 0, Loss: 1.3651
Epoch: 0, Loss: 1.4549
Epoch: 0, Loss: 1.3487
Epoch: 0, Loss: 0.8728
Epoch: 0, Loss: 1.6423
Epoch: 0, Loss: 0.5961
Epoch: 0, Loss: 1.8930
Epoch: 0, Loss: 1.5476
Epoch: 0, Loss: 1.2612
Epoch: 0, Loss: 1.1576
Epoch: 0, Loss: 1.2919
Epoch: 0, Loss: 0.7792
Epoch: 0, Loss: 1.1941
Epoch: 0, Loss: 1.6547
Epoch: 0, Loss: 0.8255
Epoch: 0, Loss: 1.0802
Epoch: 0, Loss: 1.2529
Epoch: 0, Loss: 1.1799
Epoch: 0, Loss: 0.7075
Epoch: 0, Loss: 1.5714
Epoch: 0, Loss: 0.8234
Epoch: 0, Loss: 1.8018
Epoch: 0, Loss: 0.5938
Epoch: 0, Loss: 1.8328
Epoch: 0, Loss: 0.3217
Epoch: 0, Loss: 0.9821
Epoch: 0, Loss: 1.0110
Epoch: 0, Loss: 1.4407
Epoch: 0, Loss: 0.9164
Epoch: 0, Loss: 1.1977
Epoch: 0, Loss: 1.4429
Epoch: 0, Loss: 1.3473
Epoch: 0, L

In [29]:
len(entities)

110

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import random

# Define the KnowledgeGraph dataset class
class KnowledgeGraphDataset(Dataset):
    def __init__(self, triples, entity2idx, relation2idx):
        self.triples = triples
        self.entity2idx = entity2idx
        self.relation2idx = relation2idx

    def __len__(self):
        return len(self.triples)

    def __getitem__(self, idx):
        subject, relation, object_ = self.triples[idx]
        return self.entity2idx[subject], self.relation2idx[relation], self.entity2idx[object_]

# Create a list of triples (subject, relation, object)

# Map entities and relations to numerical indices
entities = list(set([subject for subject, _, _ in triples] + [object_ for _, _, object_ in triples]))
entity2idx = {entity: idx for idx, entity in enumerate(entities)}
relations = list(set([relation for _, relation, _ in triples]))
relation2idx = {relation: idx for idx, relation in enumerate(relations)}

# Split the data into training and testing sets using random split
train_triples, test_triples = train_test_split(triples, test_size=0.3, random_state=42)

# Define the KnowledgeGraph dataset
train_dataset = KnowledgeGraphDataset(train_triples, entity2idx, relation2idx)
test_dataset = KnowledgeGraphDataset(test_triples, entity2idx, relation2idx)

# Define the dataloader for training and testing
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=2, shuffle=False)

# Set the hyperparameters
num_entities = len(entities)
num_relations = len(relations)
embedding_dim = 3
learning_rate = 0.02
num_epochs = 5

# Define the TransE model
model = TransE(num_entities, num_relations, embedding_dim)

# Define the loss function
criterion = nn.MarginRankingLoss(margin=1.0)

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    for batch in train_dataloader:
        optimizer.zero_grad()

        # Positive samples
        positive_samples = torch.stack(batch, dim=1).long()

        # Generate negative samples by corrupting the objects in positive samples
        negative_samples = positive_samples.clone()
        negative_samples[:, 2] = torch.randint(num_entities, size=(negative_samples.size(0),))

        # Forward pass
        pos_subject, pos_relation, pos_object = model(positive_samples)
        neg_subject, neg_relation, neg_object = model(negative_samples)

        # Compute the loss
        target = torch.ones(pos_subject.size(0), 1)
        loss = criterion(pos_subject + pos_relation - pos_object, neg_subject + neg_relation - neg_object, target)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        print("Epoch: {}, Loss: {:.4f}".format(epoch, loss.item()))
        
# Evaluation loop
# Evaluation loop
with torch.no_grad():
    correct_predictions = 0
    total_samples = 0

    for batch in test_dataloader:
        # Forward pass
        positive_samples = torch.stack(batch, dim=1).long()
        subject, relation, object_ = model(positive_samples)

        # Calculate distances
        distances = torch.norm(subject + relation - object_, dim=1)

        # Calculate ranks
        _, indices = torch.sort(distances)
        sorted_ranks = torch.arange(1, indices.size(0) + 1, device=indices.device)
        ranks = sorted_ranks[indices]

        # Count the number of correct predictions (lowest rank indicates highest similarity)
        correct_predictions += (ranks == 1).sum().item()
        total_samples += ranks.size(0)

    accuracy = correct_predictions / total_samples
    print("Accuracy on Test Set: {:.2f}%".format(accuracy * 100))







Epoch: 0, Loss: 1.0483
Epoch: 0, Loss: 2.0341
Epoch: 0, Loss: 1.6574
Epoch: 0, Loss: 1.1612
Epoch: 0, Loss: 1.3850
Epoch: 0, Loss: 0.1536
Epoch: 0, Loss: 1.2455
Epoch: 0, Loss: 0.6926
Epoch: 0, Loss: 0.8680
Epoch: 0, Loss: 0.9280
Epoch: 0, Loss: 1.0246
Epoch: 0, Loss: 1.7262
Epoch: 0, Loss: 1.1609
Epoch: 0, Loss: 0.4728
Epoch: 0, Loss: 1.7330
Epoch: 0, Loss: 0.9392
Epoch: 0, Loss: 1.1335
Epoch: 0, Loss: 1.5390
Epoch: 0, Loss: 1.0670
Epoch: 0, Loss: 1.1722
Epoch: 0, Loss: 0.7306
Epoch: 0, Loss: 1.5657
Epoch: 0, Loss: 0.7061
Epoch: 0, Loss: 1.0272
Epoch: 0, Loss: 0.7356
Epoch: 0, Loss: 1.5244
Epoch: 0, Loss: 1.9778
Epoch: 0, Loss: 2.2379
Epoch: 0, Loss: 1.8804
Epoch: 0, Loss: 0.7311
Epoch: 0, Loss: 0.6241
Epoch: 1, Loss: 1.3139
Epoch: 1, Loss: 1.0692
Epoch: 1, Loss: 1.1170
Epoch: 1, Loss: 0.7119
Epoch: 1, Loss: 2.0020
Epoch: 1, Loss: 0.9712
Epoch: 1, Loss: 2.2749
Epoch: 1, Loss: 1.5881
Epoch: 1, Loss: 1.3175
Epoch: 1, Loss: 1.8236
Epoch: 1, Loss: 1.5045
Epoch: 1, Loss: 1.1171
Epoch: 1, L

Epoch: 2, Loss: 0.8047
Epoch: 2, Loss: 2.2006
Epoch: 2, Loss: 1.3792
Epoch: 2, Loss: 1.2950
Epoch: 2, Loss: 1.3786
Epoch: 2, Loss: 1.6529
Epoch: 2, Loss: 1.0827
Epoch: 2, Loss: 0.9996
Epoch: 2, Loss: 1.1672
Epoch: 2, Loss: 0.1271
Epoch: 2, Loss: 0.9759
Epoch: 2, Loss: 1.0903
Epoch: 2, Loss: 0.6554
Epoch: 2, Loss: 0.9782
Epoch: 2, Loss: 0.9901
Epoch: 2, Loss: 1.1034
Epoch: 2, Loss: 0.7452
Epoch: 2, Loss: 1.3436
Epoch: 2, Loss: 1.0710
Epoch: 2, Loss: 1.4906
Epoch: 2, Loss: 0.8388
Epoch: 2, Loss: 1.8857
Epoch: 2, Loss: 1.5333
Epoch: 2, Loss: 1.2403
Epoch: 2, Loss: 1.0967
Epoch: 2, Loss: 1.7470
Epoch: 2, Loss: 1.0724
Epoch: 2, Loss: 1.3222
Epoch: 2, Loss: 1.0428
Epoch: 3, Loss: 1.6281
Epoch: 3, Loss: 0.6175
Epoch: 3, Loss: 1.1524
Epoch: 3, Loss: 0.9979
Epoch: 3, Loss: 0.4153
Epoch: 3, Loss: 1.7007
Epoch: 3, Loss: 0.7351
Epoch: 3, Loss: 1.7770
Epoch: 3, Loss: 0.6515
Epoch: 3, Loss: 0.8179
Epoch: 3, Loss: 1.6786
Epoch: 3, Loss: 0.7015
Epoch: 3, Loss: 0.6954
Epoch: 3, Loss: 1.6327
Epoch: 3, L

In [14]:
#To further analyze the performance, you may want to evaluate other metrics such as mean rank, mean reciprocal rank (MRR),
# or hits at K. These metrics provide a more comprehensive understanding of the model's performance.

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import random

# Define the KnowledgeGraph dataset class
class KnowledgeGraphDataset(Dataset):
    def __init__(self, triples, entity2idx, relation2idx):
        self.triples = triples
        self.entity2idx = entity2idx
        self.relation2idx = relation2idx

    def __len__(self):
        return len(self.triples)

    def __getitem__(self, idx):
        subject, relation, object_ = self.triples[idx]
        return self.entity2idx[subject], self.relation2idx[relation], self.entity2idx[object_]

# Create a list of triples (subject, relation, object)

# Map entities and relations to numerical indices
entities = list(set([subject for subject, _, _ in triples] + [object_ for _, _, object_ in triples]))
entity2idx = {entity: idx for idx, entity in enumerate(entities)}
relations = list(set([relation for _, relation, _ in triples]))
relation2idx = {relation: idx for idx, relation in enumerate(relations)}

# Split the data into training and testing sets using random split
train_triples, test_triples = train_test_split(triples, test_size=0.3, random_state=42)

# Define the KnowledgeGraph dataset
train_dataset = KnowledgeGraphDataset(train_triples, entity2idx, relation2idx)
test_dataset = KnowledgeGraphDataset(test_triples, entity2idx, relation2idx)

# Define the dataloader for training and testing
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=2, shuffle=False)

# Set the hyperparameters
num_entities = len(entities)
num_relations = len(relations)
embedding_dim = 3
learning_rate = 0.02
num_epochs = 5

# Define the TransE model
model = TransE(num_entities, num_relations, embedding_dim)

# Define the loss function
criterion = nn.MarginRankingLoss(margin=1.0)

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    for batch in train_dataloader:
        optimizer.zero_grad()

        # Positive samples
        positive_samples = torch.stack(batch, dim=1).long()

        # Generate negative samples by corrupting the objects in positive samples
        negative_samples = positive_samples.clone()
        negative_samples[:, 2] = torch.randint(num_entities, size=(negative_samples.size(0),))

        # Forward pass
        pos_subject, pos_relation, pos_object = model(positive_samples)
        neg_subject, neg_relation, neg_object = model(negative_samples)

        # Compute the loss
        target = torch.ones(pos_subject.size(0), 1)
        loss = criterion(pos_subject + pos_relation - pos_object, neg_subject + neg_relation - neg_object, target)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        print("Epoch: {}, Loss: {:.4f}".format(epoch, loss.item()))
        

# Evaluation loop
with torch.no_grad():
    ranks = []
    reciprocal_ranks = []

    for batch in test_dataloader:
        # Forward pass
        positive_samples = torch.stack(batch, dim=1).long()
        subject, relation, object_ = model(positive_samples)

        # Calculate distances
        distances = torch.norm(subject + relation - object_, dim=1)

        # Calculate ranks
        _, indices = torch.sort(distances)
        sorted_ranks = torch.arange(1, indices.size(0) + 1, device=indices.device)
        ranks.extend(sorted_ranks[indices].tolist())

        # Calculate reciprocal ranks
        reciprocal_ranks.extend((1.0 / sorted_ranks[indices]).tolist())

    mean_rank = sum(ranks) / len(ranks)
    mrr = sum(reciprocal_ranks) / len(reciprocal_ranks)

    print("Mean Rank on Test Set: {:.2f}".format(mean_rank))
    print("MRR on Test Set: {:.4f}".format(mrr))









Epoch: 0, Loss: 1.4078
Epoch: 0, Loss: 1.2694
Epoch: 0, Loss: 1.7255
Epoch: 0, Loss: 1.0912
Epoch: 0, Loss: 1.1723
Epoch: 0, Loss: 1.2063
Epoch: 0, Loss: 1.3286
Epoch: 0, Loss: 2.2007
Epoch: 0, Loss: 0.9012
Epoch: 0, Loss: 0.7931
Epoch: 0, Loss: 0.7897
Epoch: 0, Loss: 1.3881
Epoch: 0, Loss: 1.0467
Epoch: 0, Loss: 1.7327
Epoch: 0, Loss: 0.6158
Epoch: 0, Loss: 1.2265
Epoch: 0, Loss: 1.1626
Epoch: 0, Loss: 0.9960
Epoch: 0, Loss: 1.4792
Epoch: 0, Loss: 1.3182
Epoch: 0, Loss: 0.8824
Epoch: 0, Loss: 0.9238
Epoch: 0, Loss: 0.6497
Epoch: 0, Loss: 1.0115
Epoch: 0, Loss: 1.2743
Epoch: 0, Loss: 1.5525
Epoch: 0, Loss: 0.9292
Epoch: 0, Loss: 0.8554
Epoch: 0, Loss: 1.5606
Epoch: 0, Loss: 1.1482
Epoch: 0, Loss: 0.3184
Epoch: 1, Loss: 1.0569
Epoch: 1, Loss: 1.4920
Epoch: 1, Loss: 1.6145
Epoch: 1, Loss: 0.7857
Epoch: 1, Loss: 0.9676
Epoch: 1, Loss: 1.2957
Epoch: 1, Loss: 1.2854
Epoch: 1, Loss: 1.0150
Epoch: 1, Loss: 0.5461
Epoch: 1, Loss: 0.4794
Epoch: 1, Loss: 0.4789
Epoch: 1, Loss: 0.8009
Epoch: 1, L

In [16]:
#Great! The mean rank of 1.5 indicates that, on average, the true positive samples are ranked at the 1st or 2nd position among all the samples. This suggests that the model is performing well in terms of ranking accuracy.

#The Mean Reciprocal Rank (MRR) value of 0.75 also indicates good performance. It means that, on average, the reciprocal rank of the true positive sample is 0.75, reflecting that the correct answer is highly likely to be ranked within the top positions.

In [17]:
# Save the model to H5 file
torch.save(model.state_dict(), "model.h5")



In [18]:
# Load the model from H5 file
model = TransE(num_entities, num_relations, embedding_dim)
model.load_state_dict(torch.load("model.h5"))

<All keys matched successfully>

In [19]:
# Load the saved model
model = TransE(num_entities, num_relations, embedding_dim)
model.load_state_dict(torch.load("model.h5"))

# Set the model to evaluation mode
model.eval()

TransE(
  (entity_embeddings): Embedding(110, 3)
  (relation_embeddings): Embedding(13, 3)
)

In [20]:
def recommend_related_entities(model, entity, k=5):
    # Set the model to evaluation mode
    model.eval()

    # Get the embedding of the given entity
    entity_idx = entity2idx[entity]
    entity_embedding = model.entity_embeddings(torch.tensor([entity_idx]))

    # Calculate the distance between the given entity and all other entities
    distances = torch.norm(model.entity_embeddings.weight - entity_embedding, dim=1)

    # Get the indices of the top k closest entities
    _, indices = torch.topk(distances, k, largest=False)

    # Convert the indices to entity names
    recommendations = [entities[idx] for idx in indices]

    return recommendations

# Test the recommendation system
entity = "risk management"
recommendations = recommend_related_entities(model, entity)
print(f"Entities related to {entity}: {recommendations}")


Entities related to risk management: ['risk management', 'avoid', 'prompt list', 'scope baseline', 'monitor risk']


In [21]:
def predict_relation(model, entity1, entity2):
    # Set the model to evaluation mode
    model.eval()

    # Get the embeddings of the given entities
    entity1_idx = entity2idx[entity1]
    entity1_embedding = model.entity_embeddings(torch.tensor([entity1_idx]))
    entity2_idx = entity2idx[entity2]
    entity2_embedding = model.entity_embeddings(torch.tensor([entity2_idx]))

    # Calculate the difference between the two entity embeddings
    diff = entity2_embedding - entity1_embedding

    # Calculate the distance between the difference and all relation embeddings
    distances = torch.norm(model.relation_embeddings.weight - diff, dim=1)

    # Get the index of the closest relation
    relation_idx = torch.argmin(distances).item()

    # Convert the index to a relation name
    relation = relations[relation_idx]

    return relation

# Test the prediction function
entity1 = "project document update"
entity2 = "strategy for overall project risk"
relation = predict_relation(model, entity1, entity2)
print(f"The most likely relation between {entity1} and {entity2} is {relation}")


The most likely relation between project document update and strategy for overall project risk is document


In [30]:
# Save entities and relations to a text file
with open("entities.txt", "w") as file:
    for entity in entities:
        file.write(f"{entity}\n")

with open("relations.txt", "w") as file:
    for relation in relations:
        file.write(f"{relation}\n")


In [31]:
entity2idx = {entity: idx for idx, entity in enumerate(entities)}
import json

# Save the entity2idx dictionary to a file
with open("entity2idx.json", "w") as file:
    json.dump(entity2idx, file)
